In [1]:
import pandas as pd

df = pd.read_csv("concrete.csv", sep = ",")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cement        1030 non-null   float64
 1   slag          1030 non-null   float64
 2   ash           1030 non-null   float64
 3   water         1030 non-null   float64
 4   superplastic  1030 non-null   float64
 5   coarseagg     1030 non-null   float64
 6   fineagg       1030 non-null   float64
 7   age           1030 non-null   int64  
 8   strength      1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [2]:
df.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
y = df["strength"]
x = df.copy()
x.drop("strength", axis=1, inplace = True)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
(  
        x_train,  
        x_test,  
        y_train,  
        y_test,  
    ) = train_test_split(  
        x,  
        y,  
        test_size=0.25,  
        random_state=12453,  
    )  

In [6]:
display(x_train.shape, 
x_test.shape,
y_train.sum().round(2),
y_test.sum().round(2))

(772, 8)

(258, 8)

27681.52

9210.98

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor

In [8]:
pipeline = Pipeline(
        steps=[
            (
                "minmaxscaler",
                MinMaxScaler(),  
            ),
            (
                "mlpregressor",
                MLPRegressor(),  
            ),
        ],
    )

In [9]:
pipeline

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('mlpregressor', MLPRegressor())])

In [10]:
from sklearn.model_selection import GridSearchCV

In [30]:
# Cree una malla de búsqueda para el objecto GridSearchCV
    # con los siguientes parámetros de búesqueda:
    #   * De 1 a 8 neuronas en la capa oculta
    #   * Activación con la función `relu`.
    #   * Tasa de aprendizaje adaptativa
    #   * Momentun con valores de 0.7, 0.8 y 0.9
    #   * Tasa de aprendijzaje inicial de 0.01, 0.05, 0.1
    #   * Un máximo de 5000 iteraciones
    #   * Use parada temprana
param_grid = {
        'mlpregressor__hidden_layer_sizes': [(i,) for i in range(1,9)],  
        'mlpregressor__activation': ['relu'],  
        'mlpregressor__learning_rate': ['adaptive'],  
        'mlpregressor__momentum': [0.7,0.8,0.9],
        'mlpregressor__learning_rate_init': [0.01,0.05,0.1],  
        'mlpregressor__max_iter': [5000],  
        'mlpregressor__early_stopping': [True]  
}

In [31]:
# Especifique un objeto GridSearchCV con el pipeline y la malla de búsqueda,
    # y los siguientes parámetros adicionales:
    #  * Validación cruzada con 5 particiones
    #  * Compare modelos usando r^2
gridsearchcv = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv = 5,  
        scoring = 'r2',  
)

In [32]:
gridsearchcv.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                                       ('mlpregressor', MLPRegressor())]),
             param_grid={'mlpregressor__activation': ['relu'],
                         'mlpregressor__early_stopping': [True],
                         'mlpregressor__hidden_layer_sizes': [(1,), (2,), (3,),
                                                              (4,), (5,), (6,),
                                                              (7,), (8,)],
                         'mlpregressor__learning_rate': ['adaptive'],
                         'mlpregressor__learning_rate_init': [0.01, 0.05, 0.1],
                         'mlpregressor__max_iter': [5000],
                         'mlpregressor__momentum': [0.7, 0.8, 0.9]},
             scoring='r2')

In [38]:
y_trian_pred = gridsearchcv.best_estimator_.predict(x_train)  
y_test_pred = gridsearchcv.best_estimator_.predict(x_test)

In [43]:
from sklearn.metrics import mean_squared_error

In [48]:
mse_train = mean_squared_error(  
        y_train,  
        y_trian_pred,  
)
mse_train

56.02778899566437

In [50]:
mse_test = mean_squared_error(  
        y_test,  
        y_test_pred,  
)

mse_test

62.22052393130031